<a href="https://colab.research.google.com/github/Rkk-i8/Canalicular_excision/blob/main/Canalicular_excision2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Google Drive のマウント
from google.colab import drive
drive.mount('/content/drive')

# 必要なライブラリ
import pandas as pd
import numpy as np
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.families import Gaussian
from statsmodels.genmod.cov_struct import Exchangeable

# データ読み込み
xlsx_path = '/content/drive/Shareddrives/岩崎Dr_IgG4 deulk/涙小管切断術/Canalicular Occlusion.xlsx'
df_all = pd.read_excel(xlsx_path, sheet_name="all")

# 群分け
df_all["Group"] = df_all["Fluo_5area_12M"].apply(
    lambda x: "遷延群" if pd.notnull(x) and x >= 4 else "良好群" if pd.notnull(x) and x <= 3 else np.nan
)
df = df_all[df_all["Group"].notna()].copy()

# Rename columns to remove spaces and problematic characters for formula API
df.columns = df.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=True)
df = df.rename(columns={'Schirmer_1_pre': 'Schirmer_1_pre', 'Fluo_5area_pre': 'Fluo_5area_pre'})


# 補助変数
df["Group_bin"] = df["Group"].map({"良好群": 0, "遷延群": 1})
df["Sex_numeric"] = df["Sex"]


# 記述統計
def median_iqr(series):
    return f"{series.median():.3f} [{series.quantile(0.25):.3f}–{series.quantile(0.75):.3f}]"

summary_table = pd.DataFrame({
    "良好群": [
        median_iqr(df[df["Group"] == "良好群"]["Age"]),
        f"{(df[df['Group'] == '良好群']['Sex_numeric'].sum().astype(int))} / {len(df[df['Group'] == '良好群'])} ({100 * df[df['Group'] == '良好群']['Sex_numeric'].mean():.1f}%)",
        median_iqr(df[df["Group"] == "良好群"]["Schirmer_1_pre"]),
        median_iqr(df[df["Group"] == "良好群"]["Fluo_5area_pre"]),
        median_iqr(df[df["Group"] == "良好群"]["TM_mm_pre"]),
        median_iqr(df[df["Group"] == "良好群"]["TM_mm_12M"]),
    ],
    "遷延群": [
        median_iqr(df[df["Group"] == "遷延群"]["Age"]),
        f"{(df[df['Group'] == '遷延群']['Sex_numeric'].sum().astype(int))} / {len(df[df['Group'] == '遷延群'])} ({100 * df[df['Group'] == '遷延群']['Sex_numeric'].mean():.1f}%)",
        median_iqr(df[df["Group"] == "遷延群"]["Schirmer_1_pre"]),
        median_iqr(df[df["Group"] == "遷延群"]["Fluo_5area_pre"]),
        median_iqr(df[df["Group"] == "遷延群"]["TM_mm_pre"]),
        median_iqr(df[df["Group"] == "遷延群"]["TM_mm_12M"]),
    ]
}, index=["年齢 (歳)", "女性割合", "Schirmer1_pre", "Fluo5area_pre", "TM_mm_pre", "TM_mm_12M"])

# GEE解析
results = []
for variable in ["Age", "Sex_numeric", "Schirmer_1_pre", "Fluo_5area_pre", "TM_mm_pre", "TM_mm_12M"]:
    model = GEE.from_formula(f"{variable} ~ Group_bin", groups="Patient_NO", data=df,
                             family=Gaussian(), cov_struct=Exchangeable())
    res = model.fit(maxiter=1000) # Increase maxiter to address IterationLimitWarning
    coef = res.params["Group_bin"]
    pval = res.pvalues["Group_bin"]
    results.append((coef, pval))

summary_table["群間差（遷延群 − 良好群）"] = [f"{c:.4f}" for c, _ in results]
summary_table["p値"] = [f"{p:.3f}" if p >= 0.001 else "<0.001" for _, p in results]

# 表示
summary_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/statsmodels/genmod/generalized_estimating_equations.py:1300: IterationLimitWarning: Iteration limit reached prior to convergence
  warnings.warn("Iteration limit reached prior to convergence",


,良好群,遷延群,群間差（遷延群 − 良好群）,p値
年齢 (歳),59.000 [49.250–67.000],65.000 [45.000–79.000],35.8642,0.716
女性割合,10 / 14 (71.4%),13 / 20 (65.0%),0.0188,0.988
Schirmer1_pre,1.500 [1.000–3.000],2.500 [2.000–4.000],0.4219,0.551
Fluo5area_pre,7.500 [3.500–8.750],11.000 [8.000–13.000],4.1491,0.004
TM_mm_pre,0.131 [0.098–0.171],0.129 [0.111–0.144],-0.0024,0.824
TM_mm_12M,0.232 [0.185–0.329],0.287 [0.150–0.331],0.0035,0.841
